# This Notebook is my try to detect labels according to MI Signals

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import os
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

In [2]:
!ls /kaggle/input/mtcaic3
base_path = "/kaggle/input/mtcaic3/"

MI	   sample_submission.csv  test.csv   validation.csv
README.md  SSVEP		  train.csv


## Here I load the data as pandas dataframe to analyze the data and to get the correlation between different features

In [3]:
def load_trial_data(row, base_path='.'):
    id_num = row['id']
    if id_num <= 4800:
        dataset = 'train'
    elif id_num <= 4900:
        dataset = 'validation'
    else:
        dataset = 'test'

    eeg_path = f"{base_path}/{row['task']}/{dataset}/{row['subject_id']}/{row['trial_session']}/EEGdata.csv"
    eeg_data = pd.read_csv(eeg_path)

    trial_num = int(row['trial'])
    samples_per_trial = 1750 if row['task'] == 'SSVEP' else 2250
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial - 1
    return eeg_data.iloc[start_idx:end_idx + 1]

def load_all_trials_df(df, base_path, start, end):
    trials = []
    for i in range(start, end):
        row = df.iloc[i]
        trial_df = load_trial_data(row, base_path)
        trials.append(trial_df)

    return trials


In [4]:
train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))
validation_df = pd.read_csv(os.path.join(base_path, 'validation.csv'))
test_df = pd.read_csv(os.path.join(base_path, 'test.csv'))

In [ ]:
train_x = load_all_trials_df(train_df, base_path,0,2400)
val_x = load_all_trials_df(validation_df, base_path,0,50)
test_x = load_all_trials_df(test_df, base_path,0,50)

In [6]:


# Assume train_x is a list of DataFrames
num_samples = len(train_x)
columns = train_x[0].columns

# Initialize correlation table with zeros
correlation_table = pd.DataFrame(0.0, index=columns, columns=columns)

# Accumulate correlations
for df in train_x:
    for col1 in columns:
        for col2 in columns:
            corr = df[col1].corr(df[col2])
            if pd.notnull(corr):
                correlation_table.loc[col1, col2] += corr

# Average over all samples
correlation_table /= num_samples

# Round and print
print(correlation_table.round(3))


/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


             Time     FZ     C3     CZ     C4     PZ    PO7     OZ    PO8  \
Time        1.000  0.008  0.008 -0.043 -0.075 -0.015 -0.005 -0.002 -0.019   
FZ          0.008  1.000  0.608  0.759  0.676  0.597  0.384  0.472  0.495   
C3          0.008  0.608  1.000  0.793  0.729  0.767  0.807  0.820  0.726   
CZ         -0.043  0.759  0.793  1.000  0.903  0.857  0.673  0.763  0.794   
C4         -0.075  0.676  0.729  0.903  1.000  0.848  0.678  0.759  0.838   
PZ         -0.015  0.597  0.767  0.857  0.848  1.000  0.788  0.869  0.900   
PO7        -0.005  0.384  0.807  0.673  0.678  0.788  1.000  0.894  0.843   
OZ         -0.002  0.472  0.820  0.763  0.759  0.869  0.894  1.000  0.887   
PO8        -0.019  0.495  0.726  0.794  0.838  0.900  0.843  0.887  1.000   
AccX       -0.019  0.003  0.005  0.004  0.006  0.006  0.006  0.005  0.009   
AccY        0.032  0.004 -0.003 -0.003 -0.002 -0.003 -0.001 -0.003 -0.003   
AccZ        0.066  0.005 -0.002 -0.005 -0.005 -0.002  0.000 -0.003 -0.005   

In [6]:
# --- 3. Your data loading functions ---

def load_trial_data(row, base_path='.'):
    id_num = row['id']
    if id_num <= 4800:
        dataset = 'train'
    elif id_num <= 4900:
        dataset = 'validation'
    else:
        dataset = 'test'

    eeg_path = f"{base_path}/{row['task']}/{dataset}/{row['subject_id']}/{row['trial_session']}/EEGdata.csv"
    eeg_data = pd.read_csv(eeg_path)

    trial_num = int(row['trial'])
    samples_per_trial = 1750 if row['task'] == 'SSVEP' else 2250
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial - 1
    return eeg_data.iloc[start_idx:end_idx + 1]

def load_all_trials(df, base_path, channels, start, end):
    trials = []
    for i in range(start, end):
        row = df.iloc[i]
        trial_df = load_trial_data(row, base_path)[channels].to_numpy()
        trials.append(trial_df)

    trials = np.stack(trials)
    return trials


### The Chosen features according to the correlation table

In [7]:
MI_channels = ['C3', 'FZ', 'PO8','Time','AccX' ,  'AccY' ,  'AccZ' , 'Gyro1'  ,'Gyro2'  ,'Gyro3']

### Loading the data as numpy arrays

In [8]:
train_x = load_all_trials(train_df, base_path, MI_channels,0,2400)
val_x = load_all_trials(validation_df, base_path, MI_channels,0,50)
test_x = load_all_trials(test_df, base_path, MI_channels,0,50)

# Since load_all_trials now returns a list, you can check the number of trials loaded
print("Number of trials in train_x:", len(train_x))
print("Number of trials in val_x:", len(val_x))
print("Number of trials in test_x:", len(test_x))

# You can also inspect the shape of the first few trials to see the different lengths
print("Shape of the first trial in train_x:", train_x[0].shape)
print("Shape of the first trial in val_x:", val_x[0].shape)
print("Shape of the first trial in test_x:", test_x[0].shape)

Number of trials in train_x: 2400
Number of trials in val_x: 50
Number of trials in test_x: 50
Shape of the first trial in train_x: (2250, 10)
Shape of the first trial in val_x: (2250, 10)
Shape of the first trial in test_x: (2250, 10)


In [9]:
train_labels=pd.read_csv(base_path+"train.csv")
train_labels = train_labels[0:2400]

val_labels=pd.read_csv(base_path+"validation.csv")
val_labels = val_labels[0:50]

In [10]:
train_labels = train_labels.iloc[:,-1]
val_labels = val_labels.iloc[:,-1]

In [11]:
label_to_index = {
    "Left": 0,
    "Right": 1,
}
train_labels = [label_to_index[label] for label in train_labels]
val_labels = [label_to_index[label] for label in val_labels]


In [12]:
train_labels = torch.tensor(train_labels, dtype=torch.float32).unsqueeze(1)  # shape: [N, 1]
val_labels = torch.tensor(val_labels, dtype=torch.float32).unsqueeze(1)

In [13]:
train_x = torch.from_numpy(train_x)
val_x = torch.from_numpy(val_x)

In [17]:
train_x.shape

torch.Size([2400, 2250, 10])

### Constructing the model

In [67]:

class GRUClassifier(nn.Module):
    def __init__(self, input_dim=10, hidden_dim=32, num_layers=3, num_classes=1, bidirectional=True):
        super(GRUClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        # BatchNorm on input features (T is time, apply on F across B)
        self.input_bn = nn.BatchNorm1d(input_dim)

        self.gru = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers,
            batch_first=True,
            bidirectional=bidirectional
        )

        self.output_bn = nn.BatchNorm1d(hidden_dim * (2 if bidirectional else 1))
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), num_classes)

    def forward(self, x):
        # x: [B, T, F] → permute to [B, F, T] for BN1d over F
        x = self.input_bn(x.permute(0, 2, 1)).permute(0, 2, 1)

        out, h_n = self.gru(x)

        if self.bidirectional:
            h_n = torch.cat([h_n[-2], h_n[-1]], dim=-1)  # [B, 2H]
        else:
            h_n = h_n[-1]  # [B, H]

        h_n = self.output_bn(h_n)  # BN after GRU output
        logits = self.fc(h_n)
        return logits


In [17]:
class SequenceDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Return input sequence and label
        return self.data[idx], self.labels[idx]


train_dataset = SequenceDataset(train_x, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = SequenceDataset(val_x, val_labels)
val_loader=DataLoader(val_dataset, batch_size=32, shuffle=True)

In [42]:
model = GRUClassifier()

In [20]:
def evaluate(model, dataloader, device='cuda'):
    model.eval()
    correct = total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x_batch, y_batch in dataloader:
            x_batch = x_batch.to(device, torch.float32)
            y_batch = y_batch.to(device)

            logits = model(x_batch)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).int()

            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    accuracy = 100.0 * correct / total
    f1 = f1_score(all_labels, all_preds, average='weighted')  # or 'macro', 'micro' based on need

    return accuracy, f1


In [21]:
max_acc=0

In [ ]:

def train(model, train_loader, val_loader, num_epochs=10, lr=1e-3, device='cuda'):
    global max_acc, max_model
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for x_batch, y_batch in train_loader:
            x_batch = x_batch.to(device, torch.float32)
            y_batch = y_batch.to(device).float().view(-1, 1)
              # BCE expects float [B, 1]

            optimizer.zero_grad()
            logits = model(x_batch)  # [B, 1]
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        # Evaluate on train and val
        #train_acc, train_f1 = evaluate(model, train_loader, device)
        val_acc, val_f1 = evaluate(model, val_loader, device)
        if val_acc > max_acc:
            max_acc = val_acc
            torch.save(model.state_dict(), "best_model_mi.pt")

        print(
            f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f} | "
            #f"Train Acc: {train_acc:.2f}% - F1: {train_f1:.2f} | "
            f"Val Acc: {val_acc:.2f}% - F1: {val_f1:.2f}"
        )


train(model,train_loader,val_loader,100)

In [68]:
model_best = GRUClassifier()
model_best.load_state_dict(torch.load("/kaggle/working/best_model_mi.pt", map_location="cpu"))
model_best.to("cpu")
model_best.eval()  # optional but recommended during inference

test = torch.from_numpy(test_x).to(torch.float32)  # ensure float32
test = test.to("cpu")  # ensure it's also on CPU

logits = model_best(test)


### In this try I succesfully got 68 score in the validation data, I was able before to get 74 accuracy

In [69]:
model_best.to("cuda")
evaluate(model_best, val_loader, "cuda")

(68.0, 0.6810305958132045)